In [59]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

In [60]:
# Removing stopwords:

def remove_stopwords(sentence):
    stopwords = set(["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ])
    sentence = sentence.lower()
    words = sentence.split()
    end_sentence = ""
    for i in words:
        if i in stopwords:
            continue
        end_sentence = end_sentence + i + " "
    end_sentence = end_sentence[:-1]

    return sentence

In [61]:
temp_list = []
labels = []
labels_num = []
sentences = []
with open('SPAM text message 20170820 - Data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sentence = remove_stopwords(row[1])
        sentences.append(sentence)
        labels.append(row[0])
        if row[0] != "ham":
            labels_num.append(0)
        else:
            labels_num.append(1)

print(labels[0], sentences[0])

ham go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat...


In [62]:
PADDING = 'post'
OOV_TOKEN = "<OOV>"
TRAIN_SPLIT = 0.8
NUM_WORDS = 1000
MAXLEN = 120
EMBEDDING_DIM = 16

In [63]:
# Removing stopwords:

def remove_stopwords(sentence):
    stopwords = set(["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ])
    sentence = sentence.lower()
    words = sentence.split()
    end_sentence = ""
    for i in words:
        if i in stopwords:
            continue
        end_sentence = end_sentence + i + " "
    end_sentence = end_sentence[:-1]

    return sentence

In [64]:
# Train Test Split
def train_val_split(sentences, labels, training_split):
    train_size = int(len(sentences) * training_split)
    train_set, train_labels = sentences[:train_size], labels_num[:train_size]
    val_set, val_labels = sentences[train_size:], labels_num[train_size:]
    return train_set, train_labels, val_set, val_labels

In [65]:
train_set, train_labels, val_set, val_labels = train_val_split(sentences, labels, TRAIN_SPLIT)

print(f"There are {len(train_set)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(val_set)} sentences for validation.\n")
print(f"There are {len(val_labels)} labels for validation.")

There are 4457 sentences for training.

There are 4457 labels for training.

There are 1115 sentences for validation.

There are 1115 labels for validation.


In [66]:
# Fit a Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def fit_tokenizer(train_sentences, num_words, oov_token):
    tokenizer = Tokenizer(num_words = num_words, oov_token=oov_token)
    tokenizer.fit_on_texts(train_sentences)
    return tokenizer

In [67]:
tokenizer = fit_tokenizer(train_set, NUM_WORDS, OOV_TOKEN)
word_index = tokenizer.word_index
total_words = len(tokenizer.word_index) + 1

print(f"Vocabulary contains {len(word_index)} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")

Vocabulary contains 8031 words

<OOV> token included in vocabulary


In [68]:
# Generates array of token sequences and pads to same length
def seq_and_pad(sentences, tokenizer, padding, maxlen):
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences, padding=padding, maxlen=maxlen)
    return padded_sequences

In [69]:
train_padded_seq = seq_and_pad(train_set, tokenizer, PADDING, MAXLEN)
val_padded_seq = seq_and_pad(val_set, tokenizer, PADDING, MAXLEN)

print(f"Padded training sequences have shape: {train_padded_seq.shape}\n")
print(f"Padded validation sequences have shape: {val_padded_seq.shape}")

Padded training sequences have shape: (4457, 120)

Padded validation sequences have shape: (1115, 120)


In [70]:
# Tokenize labels
# should not be needed since we can just do ham = 0 and spam = 1
# Still should try this and see the result
def tokenize_labels(all_labels, split_labels):
    label_tokenizer = Tokenizer()
    label_tokenizer.fit_on_texts(all_labels)
    label_seq = np.array(label_tokenizer.texts_to_sequences(split_labels))
    label_seq_np = np.array([i - 1 for i in label_seq]) # Since keras starts values at 1
    return label_seq_np

In [71]:
# train_token_labels = tokenize_labels(labels, train_labels)
# val_token_labels = tokenize_labels(labels, val_labels)

# print(f"First 5 labels of the training set should look like this:\n{train_token_labels[:5]}\n")
# print(f"First 5 labels of the validation set should look like this:\n{val_token_labels[:5]}\n")
# print(f"Tokenized labels of the training set have shape: {train_token_labels.shape}\n")
# print(f"Tokenized labels of the validation set have shape: {val_token_labels.shape}\n")

In [77]:
# Making the Model
def create_model(num_words, embedding_dim, maxlen):
    tf.random.set_seed(123)
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(num_words, embedding_dim, input_length=maxlen),
        tf.keras.layers.Conv1D(128, 5, activation='relu'),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [78]:
model = create_model(total_words, EMBEDDING_DIM, MAXLEN)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           128512    
                                                                 
 conv1d (Conv1D)             (None, 116, 128)          10368     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_2 (Dense)             (None, 6)                 774       
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 139661 (545.55 KB)
Trainable params: 139661 (545.55 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [81]:
history = model.fit(train_padded_seq, np.array(train_labels), epochs = 10, validation_data=(val_padded_seq, np.array(val_labels))) 

Epoch 1/10
140/140 [==============================] - 1s 5ms/step - loss: 0.0025 - accuracy: 0.9989 - val_loss: 0.0610 - val_accuracy: 0.9857
Epoch 2/10
140/140 [==============================] - 1s 4ms/step - loss: 0.0024 - accuracy: 0.9987 - val_loss: 0.0675 - val_accuracy: 0.9874
Epoch 3/10
140/140 [==============================] - 0s 3ms/step - loss: 0.0021 - accuracy: 0.9991 - val_loss: 0.0683 - val_accuracy: 0.9830
Epoch 4/10
140/140 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 0.9991 - val_loss: 0.0706 - val_accuracy: 0.9848
Epoch 5/10
140/140 [==============================] - 0s 4ms/step - loss: 0.0021 - accuracy: 0.9989 - val_loss: 0.0732 - val_accuracy: 0.9839
Epoch 6/10
140/140 [==============================] - 0s 4ms/step - loss: 0.0017 - accuracy: 0.9991 - val_loss: 0.0777 - val_accuracy: 0.9848
Epoch 7/10
140/140 [==============================] - 1s 4ms/step - loss: 0.0019 - accuracy: 0.9989 - val_loss: 0.0846 - val_accuracy: 0.9857
Epoch 